# Workflow #2

This notebook plots bikeability per AOI and importance/centrality results for the given centrality variants.

In [ ]:
import algo.net_helper as nh
import algo.helper as h
import os.path
import geopandas as gpd
import matplotlib.pyplot as plt


# settings
aoi_names = ["at_wien", "at_zs", "at_ib", "at_no", "at_zw", "at_graz_15"]

vis_by_width = True
centrality_variants_core = [
    "centr_sbc_c2000_bp_d4",
    "centr_sbc_c4000_bp_d4",
    "centr_sbc_c7000_bp_d4",
    "centr_sbc_dec_2000_4000_bp_d4",
    "centr_sbc_dec_4000_7000_bp_d4",
    "centr_sbc_dec_2000_7000_bp_d4",
]
centrality_variants_ext = [
    "centr_ebc_c0_sp",
    "centr_ebc_c4000_bp_d4",
    "centr_febc_c4000_bp_d4",
    "centr_sbc_c4000_sp",
    "centr_sbc_c4000_bp_d4",
    "centr_pbc_c4000_bp_d4",
    # NWS
    "centr_sbc_c2000_bp_d4_nws_300",
    "centr_sbc_c2000_bp_d4_nws_600",
    "centr_sbc_c2000_bp_d4_nws_900",
    "centr_sbc_c2000_bp_d4_nws_1200",
    "centr_sbc_c2000_bp_d4_nws_1500",
    "centr_sbc_c4000_bp_d4_nws_300",
    "centr_sbc_c4000_bp_d4_nws_600",
    "centr_sbc_c4000_bp_d4_nws_900",
    "centr_sbc_c4000_bp_d4_nws_1200",
    "centr_sbc_c4000_bp_d4_nws_1500",
    "centr_sbc_c7000_bp_d4_nws_300",
    "centr_sbc_c7000_bp_d4_nws_600",
    "centr_sbc_c7000_bp_d4_nws_900",
    "centr_sbc_c7000_bp_d4_nws_1200",
    "centr_sbc_c7000_bp_d4_nws_1500"
]
centrality_variants = centrality_variants_core + centrality_variants_ext

file_suffix = ""

dir_data = "data"

mode = "bike_incwalk"
# tolerable access is determined by input network: all segments that have an index value assigned 
# (other than NULL, > 0) but have mode access set to False
access = "bicycle" 

# generated params
index_col_ft = f"index_{mode}_ft"
index_col_tf = f"index_{mode}_tf"
file = os.path.join(dir_data, f"r_<aoi_name>_edges{file_suffix}.gpkg")
centr_cols = [f"{c}_sum" for c in centrality_variants]
f_aoi = os.path.join(dir_data, "aois.gpkg")


In [ ]:
# load AOI data (full extent and core area)
aois_core = gpd.read_file(f_aoi, columns=["aoi_name", "geom"], layer="core_extent_wgs84").set_index("aoi_name")

In [ ]:
def plot_aoi(aoi, plot_core_aoi=False, plot_core_aoi_mask=True, lw=0.3):
    fn = file.replace("<aoi_name>", aoi)
    if not os.path.exists(fn):
        print(f"ERR: file '{fn}' not found.")
        return
    # check availability of AOI extent
    if aoi not in aois_core.index.values:
        print(f"ERR: AOI extent not found in AOI table ('{aoi}')")
        return
    # read available columns
    cols = nh.read_cols_from_gpkg(fn)
    ccols = []
    for cc in centr_cols:
        if cc in cols:
            ccols.append(cc)
        else:
            print(f"WARN: centrality column '{cc}' not found for AOI '{aoi}'")
    net = gpd.read_file(fn, engine='pyogrio')   
    # prepare output dirs
    pltdir = os.path.join("plots", aoi)
    if not os.path.exists(pltdir):
        os.makedirs(pltdir)
    # plot bikeability
    print("...plotting bikeability...")
    b_net = net[(net[f"access_{access}_ft"] | net[f"access_{access}_tf"])]
    b_net.loc[:,"index_vis"] = (b_net[index_col_ft] + b_net[index_col_tf]) / 2 # use mean of index_ft and index_tf
    ## iteratively plot per bikeability class -> control drawing order (best is last / on top)
    ax = b_net[~(b_net.index_vis > 0.2)].plot(color="#D7191C", linewidth=lw)
    b_net[~(b_net.index_vis > 0.4) & (b_net.index_vis > 0.2)].plot(ax=ax, color="#FDAE61", linewidth=lw)
    b_net[~(b_net.index_vis > 0.6) & (b_net.index_vis > 0.4)].plot(ax=ax, color="#FFFFBF", linewidth=lw)
    b_net[~(b_net.index_vis > 0.8) & (b_net.index_vis > 0.6)].plot(ax=ax, color="#ABDDA4", linewidth=lw)
    b_net[(b_net.index_vis > 0.8)].plot(ax=ax, color="#2B83BA", linewidth=lw)
    
    ## add core AOI extent
    h.set_plot_aoi(ax, aoi, plot_core_aoi, plot_core_aoi_mask)
    ## plot map
    h.plot_map(ax, "bikeability", pltdir, dpi=1000)
    plt.show()
    
    # plot centrality
    print("...plotting centrality...")
    for cc in ccols:
        print(cc)
        if vis_by_width:
            # visualize centrality by line width
            net["lw"] = net[cc] / net[cc].max() * 2
            net_sorted = net.sort_values(by=cc, ascending=True)
            ax = net_sorted.plot(color="#017AB8", linewidth=net_sorted["lw"]) #YlGnBu // YlOrRd // Blues
        else:
            # visualize centrality by line color
            ax = net.sort_values(by=cc, ascending=True).plot(column=cc, cmap="GnBu", linewidth=0.5) #YlGnBu // YlOrRd // Blues
        ## add core AOI extent
        h.set_plot_aoi(ax, aoi, plot_core_aoi, plot_core_aoi_mask)
        h.plot_map(ax, cc, pltdir, dpi=1000)
        plt.show()

In [ ]:
for aoi in aoi_names:
    plot_aoi(aoi, False, True)